In [1]:
import pandas as pd
import transformers
import numpy as np
import os
import json
import ast
# from samutil import SpacyTransformer
import spacy
model = spacy.load('en_core_web_sm')
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer

import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

from sklearn.utils.extmath import softmax
import numpy as np



In [53]:
df = pd.read_json('../data/main/tmdb_data_main.json')
df = df[['tmdb_id','movie','overview','genres','genres_new','popularity','release_year','vote_average','budget','revenue','poster_path']]
df = df.dropna(subset=['genres'])



genre_include = df.explode('genres').genres.value_counts().keys()[0:12]

def genres_check(film_genres):
    
    for genre in film_genres:
        if genre not in genre_include:
            return False
    return True

df['genres_valid'] = df['genres'].apply(genres_check)

df = df[df.genres_valid ==True]

df.shape

genre_include = list(genre_include)

genre_include.sort()

genre_include



def multi_label_prep(film_genres):
    prep = []
    for genre in genre_include:
        if genre in film_genres:
            prep.append(1)
        else:
            prep.append(0)
            
    return prep
            
df['labels'] = df['genres'].apply(multi_label_prep)

(15001, 12)

,genres,labels
0,"[Horror, Thriller]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
2,"[Drama, Crime]","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0]"
4,"[Science Fiction, Drama]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]"
5,[Horror],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
6,"[Action, Comedy]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
19615,"[Comedy, Family, Drama]","[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]"
19622,"[Crime, Adventure]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
19638,[Animation],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
19639,"[Adventure, Comedy]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
